In [2]:
from pathlib import Path
from itertools import product
import h5py
import numpy as np
import einops as eo
from matplotlib import pyplot as plt
plt.rcParams['text.usetex'] = False
import torch
import torchkbnufft as tkbn
from functorch import vmap

from src import preprocessing as pre
from src import io_utils as iou
from src import reconstruction as recon
from src import computation as comp
from src.density_compensation import *
from src.coil_sensitivity_estimation import *

from tqdm import tqdm
from struct import pack
from nibabel import load

from src.twix_metadata_def import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
args = recon.CAPTURE_VarW_NQM_DCE_PostInj_Args(
        datFileLocation=Path(
        '/data/anlab/PET_MOCO/PETMRdata/CAPTURE_DCE/ONC-DCE-004/meas_MID00165_FID04589_Abd_CAPTURE_FA15_Dyn.dat'),
        last_slice=80
        )
import re
which_scan=-1
datFileLocation=args.datFileLocation
offset = 0
prisma_header = np.fromfile(
    datFileLocation, dtype=prisma_header_type, count=1)[0]
offset = prisma_header.itemsize

meas_list = []
for k in range(prisma_header['numberOfScansInFile']):
    meas = np.fromfile(datFileLocation, dtype=meas_type,
                       count=1, offset=offset)[0]
    offset += 152
    meas_list.append(meas)

measOffset, measLength = meas_list[which_scan]['measOffset'], meas_list[which_scan]['measLength']
offset = measOffset
nProtHeaderLen = np.fromfile(
    datFileLocation, dtype='<u4', count=1, offset=offset)[0]
offset += 4
AAA = np.fromfile(datFileLocation, dtype='<u1',
                  count=nProtHeaderLen-4, offset=int(offset))
AAA = ''.join(map(chr, AAA))
buffer = re.sub(r'\n\s*\n', '', AAA)


In [7]:
buffer[:buffer.find('ASCCONV')]

'\x06\x00\x00\x00Config\x00\x11z\x01\x00<XProtocol> \n{  <ParamMap.""> \n  {\n    <ParamMap."PARC"> \n    {      <ParamMap."HARDWARE"> \n      {        <ParamString."CPUCount">  { "12"  }\n        <ParamString."MaxMemory">  { "42059"  }\n        <ParamString."MaxRawObjectSize">  { "31643"  }\n        <ParamString."MaxRawObjectSizePrioRecon">  { "1200"  }\n        <ParamString."ProtectedSize">  { "1250"  }\n        <ParamString."RAIDSize">  { "0"  }\n        <ParamString."PDSFastSize">  { "0"  }\n        <ParamString."PDSSaveSize">  { "0"  }\n        <ParamString."NumberOfGPUs">  { "0"  }\n      }\n      <ParamMap."RECOMPOSE"> \n      {        <ParamString."PatientID">  { "xxxxxxxxxxx"  }\n        <ParamString."PatientBirthDay">  { "xxxxxxxx"  }\n        <ParamLong."PatientSex">  { 2  }\n        <ParamString."tPatientName">  { "xxxxxxxxxxx"  }\n        <ParamLong."SubProtocolIndex">  { }\n        <ParamString."PatientLOID">  { "22.0.22713448"  }\n        <ParamString."StudyLOID">  { "22

In [5]:
from src.io_utils import *

reASCCONV = re.compile(
    r'### ASCCONV BEGIN[^\n]*\n(.*)\s### ASCCONV END ###', re.DOTALL)
# print(f'buffer = {buffer[0:10]}')
# import pdb; pdb.set_trace()

ascconv = reASCCONV.search(buffer)
print(f'ascconv = {ascconv}')


ascconv = <re.Match object; span=(110477, 1122644), match='### ASCCONV BEGIN object=MrCardioViewLabelSuffixL>


In [ ]:
if ascconv is not None:
    prot = parse_ascconv(ascconv.group(0))
else:
    # prot = AttrDict()
    prot = dict()

xprot = reASCCONV.split(buffer)
# print(f'xprot = {xprot[0][0:10]}')
if xprot is not None:
    xprot = ''.join([found for found in xprot])
    prot2 = parse_xprot(xprot)

    prot.update(prot2)


In [8]:
import mapvbvd
twixObj = mapvbvd.mapVBVD(datFileLocation)

pymapVBVD version 0+untagged.2.gadb90b6.dirty
Software version: VD


Scan 1/2, read all mdhs:   0%|          | 0.00/14.7M [00:00<?, ?B/s]

Scan 2/2, read all mdhs:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

In [14]:
twixObj[1]

AttrDict({'hdr': ***twix_hdr***
Top level data structures: 
Config
Dicom
Meas
MeasYaps
Phoenix
Spice
, 'image': ***twix_map_obj***
File: /data/anlab/PET_MOCO/PETMRdata/CAPTURE_DCE/ONC-DCE-004/meas_MID00165_FID04589_Abd_CAPTURE_FA15_Dyn.dat
Software: vd
Number of acquisitions read 96000
Data size is [ 640,  15,3000,  32,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
    1,   1]
Squeezed data size is [320,15,3000,32] (['Col', 'Cha', 'Lin', 'Par'])
NCol = 640
NCha = 15
NLin  = 3000
NAve  = 1
NSli  = 1
NPar  = 32
NEco  = 1
NPhs  = 1
NRep  = 1
NSet  = 1
NSeg  = 1
NIda  = 1
NIdb  = 1
NIdc  = 1
NIdd  = 1
NIde  = 1})